#**Ahmed Hassan Abbas**

In [36]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=27b27ef275ffd5e8eca2a0492bc7afc6744d44103b1bad65f7bf1702f3066a48
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
!pip install langchain langchain_community langchain_google_genai chromadb gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.4 MB/s eta 0:00:00
 

In [ ]:
!unzip KOTOB.zip

Archive:  KOTOB.zip
   creating: KOTOB/
  inflating: KOTOB/DOAA.json         
  inflating: KOTOB/bukhari.json      
  inflating: KOTOB/athkar.json       
  inflating: KOTOB/aladab_almufrad.json  
  inflating: KOTOB/ahmed.json        
  inflating: KOTOB/abudawud.json     
  inflating: KOTOB/tirmidhi.json     
  inflating: KOTOB/nasai.json        
  inflating: KOTOB/muslim.json       
  inflating: KOTOB/malik.json        
  inflating: KOTOB/ibnmajah.json     
  inflating: KOTOB/darimi.json       


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
import os
import json
import requests
import gradio as gr
import re

In [ ]:
def normalize_arabic(text: str) -> str:
    if not text:
        return ""
    text = re.sub(r'[\u0610-\u061A\u064B-\u065F\u06D6-\u06ED]', '', text)
    text = text.replace('\u200f','').replace('\u200e','')
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

In [ ]:
# ===================================
# 2) Loading data, metadata
# ===================================
documents = []
files_to_load = ['muslim.json', 'bukhari.json', 'malik.json', 'ahmed.json']

for file_name in files_to_load:
    file_path = os.path.join(file_name)
    try:
        if file_name.endswith('.json'):
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            if 'hadiths' in data:
                for hadith in data['hadiths']:
                    if 'arabic' in hadith and hadith['arabic']:
                        content = normalize_arabic(hadith['arabic'])
                        metadata = {
                            "source": file_name,
                            "id": hadith.get('id'),
                            "bookId": hadith.get('bookId'),
                            "chapterId": hadith.get('chapterId')
                        }
                        documents.append(Document(page_content=content, metadata=metadata))
            print(f"Loaded from {file_name}")

        elif file_name.endswith('.txt'):
            loader = TextLoader(file_path)
            loaded_docs = loader.load()
            for doc in loaded_docs:
                doc.metadata["source"] = file_name
            documents.extend(loaded_docs)
            print(f"Loaded {len(loaded_docs)} from {file_name}")

        else:
            print(f"Skipping unsupported: {file_name}")
    except Exception as e:
        print(f"Error in {file_name}: {e}")

print(f"\n Total documents loaded: {len(documents)}")


Loaded from muslim.json
Loaded from bukhari.json
Loaded from malik.json
Loaded from ahmed.json

 Total documents loaded: 17970


In [72]:
# ===================================
# 3) Create  Embeddings , VectorStore
# ===================================
from langchain.embeddings import HuggingFaceEmbeddings

# embeddings = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001",
#     google_api_key=userdata.get('GEMINI_API_KEY')
# )

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")


text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
split_documents = text_splitter.split_documents(documents)

print(f"Original docs: {len(documents)}")
print(f"Split docs: {len(split_documents)}")

Original docs: 17970
Split docs: 19185


In [ ]:
vectorstore = Chroma.from_documents(
    documents=split_documents,
    embedding=embeddings,
    persist_directory="./chroma_db"
)
vectorstore.persist()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/tmp/ipython-input-2668280718.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
# ===================================
# 4) OpenRouter API connection
# ===================================
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

def call_openrouter_api(prompt):
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json"
        },
        json={
            "model": "openai/gpt-oss-120b",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "max_tokens": 900
        }
    )
    data = response.json()
    try:
        return data['choices'][0]['message']['content']
    except:
        return response.json()

In [ ]:
call_openrouter_api("Hello")

'Hello! How can I assist you today?'

In [ ]:
# ===================================
# 5) Re-ranking
# ===================================
def rerank_results(query, docs, top_k=5):
    scores = []
    for doc in docs:
        check_prompt = f"""السؤال: {query}
النص: {doc.page_content}

هل النص أعلاه يجيب على السؤال بشكل مباشر؟ أجب فقط بـ نعم أو لا.
"""
        ans = call_openrouter_api(check_prompt)
        score = 1 if "نعم" in ans else 0
        scores.append((doc, score))
    sorted_docs = sorted(scores, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in sorted_docs[:top_k]]


In [67]:
from langchain_community.utilities import GoogleSerperAPIWrapper
import os

SERPER_API_KEY = userdata.get('SERPER_API_KEY')

In [71]:
# ===================================================
# 6) Searching with filtering by metadata, Re-ranking
# ===================================================
def ask_question_with_retrieval(query, book_filter="All Books"):
    search_kwargs = {"k": 50}
    if book_filter != "All Books":
        search_kwargs["filter"] = {"source": book_filter}

    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs=search_kwargs)
    initial_docs = retriever.get_relevant_documents(query)

    top_docs = rerank_results(query, initial_docs, top_k=5)

    context = "\n".join([f"Document from Vectorstore:\n{doc.page_content}" for doc in top_docs])

    scraped_content = ""
    scraped_source_url = ""

    if SERPER_API_KEY:
        print("Performing web search to supplement retrieval...")
        try:
            search = GoogleSerperAPIWrapper(serper_api_key=SERPER_API_KEY)
            search_results = search.results(query)
            web_context = ""

            if 'organic' in search_results:
                for result in search_results['organic']:
                    if result.get('link') and "shamela.ws" in result['link']:
                        print(f"Found trusted source: {result['link']}. Attempting to scrape.")
                        scrape_url = result['link']
                        scrape_response = requests.post(
                            url="https://serper.dev/api/scrape",
                            headers={
                                "X-API-KEY": SERPER_API_KEY,
                                "Content-Type": "application/json"
                            },
                            json={"url": scrape_url}
                        )
                        if scrape_response.status_code == 200:
                            scrape_data = scrape_response.json()
                            if 'article' in scrape_data and scrape_data['article']:
                                scraped_content = normalize_arabic(scrape_data['article'])
                                scraped_source_url = scrape_url
                                print("Successfully scraped content from trusted source.")
                                break
                            else:
                                print("Scrape API returned no article content.")
                        else:
                            print(f"Scrape API failed with status code: {scrape_response.status_code}")
                            print(f"Scrape API response: {scrape_response.text}")

                    web_context += f"Web Search Result from {result.get('title')}:\n{result.get('snippet')}\n\n"

            if scraped_content:
                context = f"Scraped Content from {scraped_source_url}:\n{scraped_content}\n\n" + context
                print("Scraped content incorporated into context.")
            elif web_context:
                context = context + "\n\n" + web_context
                print("General web search results incorporated.")
            else:
                print("Web search returned no relevant results.")

        except Exception as e:
            print(f"Web search or scraping failed: {e}")
    else:
        print("SERPER_API_KEY not available, skipping web search and scraping.")


    prompt = f"""
أنت وكيل مُتخصِّص في علم الحديث النبوي ومصادر المعلومات العامة. مهمتك:
1) أجب على سؤال المستخدم بدقة وبوضوح باللغة العربية.
2) استعمل فقط المعلومات الواردة في المقاطع (context) التالية عند تكوين الإجابة. لا تختلق مصادر جديدة.
3) عندما تذكر معلومات من أي مقطع من Vectorstore ضع الاستشهاد بصيغة مربعات مثل: [المصدر | رقم الحديث | من اي باب  | حالة الحديث صحيح او حسن او ضعيف او مكذوب ].
4) عندما تذكر معلومات من أي مقطع من Web Search Result اذكر المصدر بصيغة مربعات مثل: [موقع: عنوان الموقع].
5) عندما تذكر معلومات من أي مقطع من Scraped Content اذكر المصدر بصيغة مربعات مثل: [المصدر المُستَخرَج: عنوان URL].
6) إذا لم تكفِ المقاطع (من Vectorstore أو Web Search أو Scraped Content) للإجابة بصراحة اعترف بنقص الدليل وقل (لا أمتلك دليلاً كافياً في المصادر المتاحة).
7) لا تُقدّم فتوى دينية جديدة — إن كان السؤال فقهيّا ادعُ المستخدم إلى الرجوع لعلماء متخصصين بعد عرض دلائل النص.
8) اذكر نص الحديث عربيًا (إن اقتبست مباشرة من المقاطع من Vectorstore أو Scraped Content إذا كان حديثًا) بين علامتي اقتباس مزدوجة "" ثم الاستشهاد المناسب.
9) النهاية: أدرج قائمة قصيرة "المصادر المستخدمة" بترتيب الاستشهادات، **مع التفرقة بين مصادر Vectorstore ومصادر Web Search ومصادر Scraped Content عن طريق تسمية كل قسم بوضوح (مثلاً: مصادر من قاعدة الأحاديث, مصادر من البحث عبر الويب, مصادر من المحتوى المُستَخرَج).**

الآن أدناه المقاطع (context) المستخرجة من كتب الأحاديث والبحث عبر الويب والمحتوى المُستَخرَج:
{context}

السؤال: {query}

أجب الآن باتباع التعليمات أعلاه.
"""
    answer = call_openrouter_api(prompt)
    return {"result": answer, "source_documents": top_docs, "scraped_source": scraped_source_url}


In [69]:

books_list = ["All Books"] + files_to_load

def chatbot_interface(user_message, selected_book, history):
    if not user_message.strip():
        return history + [["", "Please enter a question."]]

    resp = ask_question_with_retrieval(user_message, selected_book)
    answer = resp["result"]

    if "Web Search Result" in answer or (len(resp["source_documents"]) < 5 and SERPER_API_KEY):
         answer += "\n\n*(Note: Web search was used to supplement the response.)*"

    history = history + [[user_message, answer]]
    return history

with gr.Blocks() as iface:
    gr.Markdown("# Hadith Chatbot")

    with gr.Row():
        selected_book = gr.Dropdown(choices=books_list, value="All Books", label="Select Book")

    chatbot = gr.Chatbot(label="Conversation")
    msg = gr.Textbox(placeholder="Type your message here...", label="Your Message")
    clear = gr.Button("Clear Chat")

    msg.submit(chatbot_interface, [msg, selected_book, chatbot], chatbot)
    clear.click(lambda: None, None, chatbot)

iface.launch(inline=True)

/tmp/ipython-input-1457770719.py:22: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f835def62c6ee18c4b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [70]:
test_query_2 = "ما هو حكم ترك الصلاة؟"
print(f"Testing query: {test_query_2}")
response_2 = ask_question_with_retrieval(test_query_2)

print("\nResponse from the chatbot:")
print(response_2['result'])
print("\nSource documents (from vectorstore):")
for i, doc in enumerate(response_2['source_documents']):
    print(f"Document {i+1}:")
    print(f"  Content: {doc.page_content}")
    print(f"  Source: {doc.metadata.get('source', 'N/A')}")
    print(f"  ID: {doc.metadata.get('id', 'N/A')}")
    print(f"  Book ID: {doc.metadata.get('bookId', 'N/A')}")
    print(f"  Chapter ID: {doc.metadata.get('chapterId', 'N/A')}")

Testing query: ما هو حكم ترك الصلاة؟


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Performing web search to supplement retrieval...
General web search results incorporated.

Response from the chatbot:
**حكم ترك الصلاة في الإسلام**

1. **الاجتهاد السائد (الإجماع)**
   - أغلب العلماء يرون أن من ترك الصلاة **جاحدًا لوجوبها** يُعدُّ كافرًا. وهذا ما يُستند إليه في الفتاوى الصادرة عن كبار العلماء مثل الشيخ ابن باز، حيث يذكر أن «من ترك الصلاة فهو كافر وإن كان غير جاحد لها» ويستند إلى قول النبي ﷺ في الحديث الصحيح *«العهد الذ …»* (الحديث يُستدل به في الفقه).  
   - **[موقع: حكم تارك الصلاة عمداً تساهلاً وحكم تكفيره - موقع الشيخ ابن باز]**  
   - **[موقع: ما حكم تارك الصلاة؟ - موقع الشيخ ابن باز]**  
   - **[موقع: حكم الصلاة وأهميتها - موقع الشيخ ابن باز]**  

2. **الآراء المخالفة (عدم تكفيره)**
   - **المالكي والشافعي**: يرى كلٌّ منهما أن ترك الصلاة لا يُكفر صاحبه، بل يُعاقب بالحد (الإعدام) إذا كان تركًا متعمدًا.  
     - **[موقع: عقوبة تارك الصلاة في الدنيا والآخرة - إسلام ويب]**  
   - **أبو حنيفة**: يقرّ أن من ترك الصلاة يُحجز (يُسجن) حتى يُصلِّي، ولا يُكفر.  
     - **[مو